In [1]:
import requests
import pandas as pd
import numpy as np
import folium
import json

In [2]:
grant_data = pd.read_csv('P3_GrantExport.csv', sep=';',
                         usecols = ['Institution','University', 
                                    'Approved Amount'])
grant_data.head()

,Institution,University,Approved Amount
0,NaN,Nicht zuteilbar - NA,11619.00
1,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
2,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
3,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
4,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


After some exploring with the data, we realize that entries with the following universtiy names should be excluded from our analysis -  

(1) 'Nicht zuteilbar - NA': 
This means Not Applicable and therefore doesn't make sense to assign a geolocation for such entries

(2)'NPO (Biblioth., Museen, Verwalt.) - NPO': 
This means Non Profit organizations, since no further information is available, we skip analyzing these entries

(3)'Firmen/Privatwirtschaft - FP': 
This refers to private research bodies, no further information available to determine geolocation, therefore we skip analyzing these entries

We also need to omit the entries that do not have any entry for the university name.



In [3]:
#Cleaning the data by removing entries with the above criteria

grant_data_cleaned = grant_data[(grant_data['Approved Amount']!='data not included in P3') & 
                                (grant_data['University']!='Nicht zuteilbar - NA') &
                                (grant_data['University']!='NPO (Biblioth., Museen, Verwalt.) - NPO') &
                                (grant_data['University']!='Firmen/Privatwirtschaft - FP') &
                                grant_data['University'].notnull()]


#Converting the datatype of 'Approved Amount' from string to float

grant_data_cleaned['Approved Amount'] = grant_data_cleaned['Approved Amount'].astype(float)

/Users/akhileshgotmare/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# total of all the amounts assigned to universities
all_money = grant_data_cleaned['Approved Amount'].sum()

#'Group by' operation performed on this data with respect to the univesities, these values are 
# summed for each university and then sorted as per the 'Approved Amount' attribute

data_by_university = grant_data_cleaned.groupby('University').sum().sort_values('Approved Amount', ascending=False)
data_by_university.head()

,Approved Amount
University,
Université de Genève - GE,1.838237e+09
Universität Zürich - ZH,1.826843e+09
ETH Zürich - ETHZ,1.635597e+09
Universität Bern - BE,1.519373e+09
Universität Basel - BS,1.352251e+09


In [6]:
# Defining function that will return the name of the canton
# takes as input the name of the university and returns the canton it belongs to as per 
# the geonames database

def get_canton_name(univ):
    
    # Split the string at the '-' to obtain the name of the university before the '-'
    univ_splitted=univ.split(' - ') 
    
    # We convert the German and French words for 'University' to the English one for better
    # search results
    
    if univ_splitted[0].find('Universität') != -1:
        univ_splitted.append(univ_splitted[0].replace('Universität', 'University'))
    if univ_splitted[0].find('Université') != -1:
        univ_splitted.append(univ_splitted[0].replace('Université', 'University'))
    
    
    # For each university we generate queries with the requests.get
    # and pass parameters as described in geonames documentation.
    # We consider the result from the API only if 
    # the 'fcodeName' attribute is either 'university'
    # or 'bus stop', else we return "No entry in DB".
    
    for uni in univ_splitted:
        payload = {'q': uni, 'country': 'CH', 'username':'akhilesh1412'}
        r = requests.get('http://api.geonames.org/searchJSON', params=payload)
        result = r.json()['geonames']
        for entry in result: 
            if entry['fcodeName'] == 'university' or entry['fcodeName'] == 'bus stop':
                return entry['adminCode1']
    return "No Entry in DB"

In [7]:
# We go over the data_by_university dataframe and keep track of the running total of the 
# approved amount of research funding. We also store the canton name for these universities
# in a data series. We break the loop when this running total exceeds 95% of 
# the gross total amount 

sum = 0
cantons = pd.Series()
for index, row in data_by_university.iterrows():
    sum += row['Approved Amount']
    print(index)
    
    result=get_canton_name(index)
    cantons.loc[index]=result
    pd.Series
    if sum >= 0.95*all_money:
        break

Université de Genève - GE
Universität Zürich - ZH
ETH Zürich - ETHZ
Universität Bern - BE
Universität Basel - BS
Université de Lausanne - LA
EPF Lausanne - EPFL
Université de Fribourg - FR
Université de Neuchâtel - NE
Paul Scherrer Institut - PSI
Università della Svizzera italiana - USI
Eidg. Anstalt für Wasserversorgung - EAWAG
Universität St. Gallen - SG
Eidg. Material und Prüfungsanstalt - EMPA


In [8]:
cantons

Université de Genève - GE                                 GE
Universität Zürich - ZH                                   ZH
ETH Zürich - ETHZ                                         ZH
Universität Bern - BE                                     BE
Universität Basel - BS                                    BS
Université de Lausanne - LA                               VD
EPF Lausanne - EPFL                                       VD
Université de Fribourg - FR                               FR
Université de Neuchâtel - NE                              NE
Paul Scherrer Institut - PSI                              AG
Università della Svizzera italiana - USI      No Entry in DB
Eidg. Anstalt für Wasserversorgung - EAWAG    No Entry in DB
Universität St. Gallen - SG                   No Entry in DB
Eidg. Material und Prüfungsanstalt - EMPA                 ZH
dtype: object

As is evident, only 14 universities in Switzerland seem to have about 95% of the research funding (another instance of power law). Since our function is unable to determine the cantons of three universities, we manually complete the job. 

In [9]:
cantons.loc['Università della Svizzera italiana - USI']='TI' #Ticino canton
cantons.loc['Eidg. Anstalt für Wasserversorgung - EAWAG']='ZH' #zurich canton
cantons.loc['Universität St. Gallen - SG']='SG' #St. Gallen canton
cantons

Université de Genève - GE                     GE
Universität Zürich - ZH                       ZH
ETH Zürich - ETHZ                             ZH
Universität Bern - BE                         BE
Universität Basel - BS                        BS
Université de Lausanne - LA                   VD
EPF Lausanne - EPFL                           VD
Université de Fribourg - FR                   FR
Université de Neuchâtel - NE                  NE
Paul Scherrer Institut - PSI                  AG
Università della Svizzera italiana - USI      TI
Eidg. Anstalt für Wasserversorgung - EAWAG    ZH
Universität St. Gallen - SG                   SG
Eidg. Material und Prüfungsanstalt - EMPA     ZH
dtype: object

In [11]:
# We append the pandas series with canton names to our previous dataframe
data_by_university['Canton'] = cantons
data_by_university.dropna(inplace = True)
data_by_university

,Approved Amount,Canton
University,,
Université de Genève - GE,1.838237e+09,GE
Universität Zürich - ZH,1.826843e+09,ZH
ETH Zürich - ETHZ,1.635597e+09,ZH
Universität Bern - BE,1.519373e+09,BE
Universität Basel - BS,1.352251e+09,BS
Université de Lausanne - LA,1.183291e+09,VD
EPF Lausanne - EPFL,1.175316e+09,VD
Université de Fribourg - FR,4.575262e+08,FR
Université de Neuchâtel - NE,3.832046e+08,NE


In [12]:
# In this resulting dataframe, we group the entries by 'Canton', sum all the corresponding
# amounts for each canton and finally sorth the entries as per thes eamounts
data_by_canton = data_by_university.groupby('Canton').sum().sort_values('Approved Amount', ascending=False)
data_by_canton.reset_index(inplace = True)
data_by_canton

,Canton,Approved Amount
0,ZH,3.595633e+09
1,VD,2.358607e+09
2,GE,1.838237e+09
3,BE,1.519373e+09
4,BS,1.352251e+09
5,FR,4.575262e+08
6,NE,3.832046e+08
7,AG,1.152690e+08
8,TI,8.497055e+07
9,SG,6.919495e+07


In [15]:
# Next we include all the cantons that aren't considered so far, we assign the approved amounts
# for these to be zero
with open("ch-cantons.topojson.json") as geo_file:
    geometry =json.load(geo_file)
for canton in geometry['objects']['cantons']['geometries']:
    if not data_by_canton["Canton"].isin([canton['id']]).values.any():
        data_by_canton = data_by_canton.append({'Canton': canton['id'], 'Approved Amount': 0}, 
                                               ignore_index=True)
        
# For better readibility and understanding on the Choropleth map, we indicate the
# approved amount in millions

data_by_canton['Approved Amount'] = data_by_canton['Approved Amount']/1000000
data_by_canton

,Canton,Approved Amount
0,ZH,3595.633032
1,VD,2358.606671
2,GE,1838.237308
3,BE,1519.373035
4,BS,1352.251464
5,FR,457.526170
6,NE,383.204582
7,AG,115.268969
8,TI,84.970555
9,SG,69.194954


In [17]:
swiss_map = folium.Map(location=[47, 8], zoom_start=8)
swiss_map.choropleth(geo_path="ch-cantons.topojson.json", 
                     data=data_by_canton,
                     columns=['Canton', 'Approved Amount'],
                     threshold_scale = [0, 50, 250, 1000, 2000, 3000],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='PuRd',
                     legend_name = 'Research funding'
                    )
swiss_map.save('map.html')

In [75]:
swiss_map.choropleth??